Импорты

In [80]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_absolute_error, mean_squared_error, r2_score

Загрузка датасета классификации

In [81]:
df_heart = pd.read_csv('heart.csv')
X_cls = df_heart.drop('target', axis=1)
y_cls = df_heart['target']

Разделение на тестовые и тренировочные

In [82]:
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=42, stratify=y_cls
)

Загрузка датасета регрессии

In [83]:
df_energy = pd.read_csv('energy_efficiency_data.csv')
X_reg = df_energy.drop(['Heating_Load', 'Cooling_Load'], axis=1)
y_reg = df_energy['Heating_Load']

Определение признаков

In [84]:
num_features = ['Relative_Compactness', 'Surface_Area', 'Wall_Area', 'Roof_Area', 'Overall_Height']
cat_features = ['Orientation', 'Glazing_Area', 'Glazing_Area_Distribution']

Разделение на тестовые и тренировочные

In [85]:
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

Линейная регрессия

In [86]:
class MyLinearRegressor:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.lr = learning_rate
        self.n_iter = n_iterations
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        X = np.asarray(X, dtype=np.float64)
        y = np.asarray(y, dtype=np.float64)
        n_samples, n_features = X.shape

        self.weights = np.zeros(n_features)
        self.bias = 0.0

        for _ in range(self.n_iter):
            y_pred = X @ self.weights + self.bias
            dw = (1 / n_samples) * X.T @ (y_pred - y)
            db = (1 / n_samples) * np.sum(y_pred - y)
            self.weights -= self.lr * dw
            self.bias -= self.lr * db
        return self

    def predict(self, X):
        X = np.asarray(X, dtype=np.float64)
        return X @ self.weights + self.bias

Логическая регрессия

In [87]:
class MyLogisticClassifier:
    def __init__(self, learning_rate=0.1, n_iterations=1000):
        self.lr = learning_rate
        self.n_iter = n_iterations
        self.W = None
        self.classes_ = None
        self.label_to_index = None

    def _softmax(self, z):
        z_stable = z - np.max(z, axis=1, keepdims=True)
        exp_z = np.exp(z_stable)
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def fit(self, X, y):
        X = np.asarray(X, dtype=np.float64)
        y = np.asarray(y)
        n_samples, n_features = X.shape

        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)
        self.label_to_index = {label: i for i, label in enumerate(self.classes_)}

        y_onehot = np.zeros((n_samples, n_classes))
        for i, label in enumerate(y):
            y_onehot[i, self.label_to_index[label]] = 1

        Xb = np.hstack([np.ones((n_samples, 1)), X])
        self.W = np.zeros((n_features + 1, n_classes))

        for _ in range(self.n_iter):
            logits = Xb @ self.W
            probs = self._softmax(logits)
            grad = (1 / n_samples) * Xb.T @ (probs - y_onehot)
            self.W -= self.lr * grad
        return self

    def predict_proba(self, X):
        X = np.asarray(X, dtype=np.float64)
        Xb = np.hstack([np.ones((X.shape[0], 1)), X])
        logits = Xb @ self.W
        return self._softmax(logits)

    def predict(self, X):
        probs = self.predict_proba(X)
        class_indices = np.argmax(probs, axis=1)
        return self.classes_[class_indices]

Метрики для классификации

In [88]:
def evaluate_classification(y_true, y_pred):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'F1': f1_score(y_true, y_pred, average='weighted'),
        'ROC-AUC': roc_auc_score(y_true, y_pred),
    }

Метрики для регрессии

In [89]:
def evaluate_regression(y_true, y_pred):
    return {
        'MAE': mean_absolute_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'R2': r2_score(y_true, y_pred)
    }

Запуск базовой модели логической регрессии sklearn (классификация)

In [90]:
sk_log_base = LogisticRegression(max_iter=10000, random_state=42)
sk_log_base.fit(X_train_cls, y_train_cls)
y_pred_sk_log = sk_log_base.predict(X_test_cls)

Запуск базовой кастомной модели логической регрессии (классификация)

In [91]:
my_log_base = MyLogisticClassifier(learning_rate=0.1, n_iterations=2000)
my_log_base.fit(X_train_cls.values, y_train_cls.values)
y_pred_my_log = my_log_base.predict(X_test_cls.values)

Вывод метрик классификации

In [92]:
print("Sklearn:", evaluate_classification(y_test_cls, y_pred_sk_log))
print("Custom :", evaluate_classification(y_test_cls, y_pred_my_log))

Sklearn: {'Accuracy': 0.8146341463414634, 'F1': 0.811967851371656, 'ROC-AUC': 0.8119047619047619}
Custom : {'Accuracy': 0.624390243902439, 'F1': 0.5806587275712194, 'ROC-AUC': 0.6323809523809523}


Запуск базовой модели линейной регрессии sklearn (регрессия)

In [93]:
sk_lin_base = LinearRegression()
sk_lin_base.fit(X_train_reg, y_train_reg)
y_pred_sk_lin = sk_lin_base.predict(X_test_reg)

Запуск кастомной базовой модели линейной регрессии (регрессия) 

In [94]:
my_lin_base = MyLinearRegressor(learning_rate=0.001, n_iterations=2000)
my_lin_base.fit(X_train_reg.values, y_train_reg.values)
y_pred_my_lin = my_lin_base.predict(X_test_reg.values)

C:\Users\Алексей\AppData\Local\Temp\ipykernel_3192\3091210259.py:18: RuntimeWarning: overflow encountered in matmul
  dw = (1 / n_samples) * X.T @ (y_pred - y)
c:\Users\Алексей\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\Алексей\AppData\Local\Temp\ipykernel_3192\3091210259.py:18: RuntimeWarning: invalid value encountered in matmul
  dw = (1 / n_samples) * X.T @ (y_pred - y)
C:\Users\Алексей\AppData\Local\Temp\ipykernel_3192\3091210259.py:20: RuntimeWarning: invalid value encountered in subtract
  self.weights -= self.lr * dw
C:\Users\Алексей\AppData\Local\Temp\ipykernel_3192\3091210259.py:21: RuntimeWarning: invalid value encountered in scalar subtract
  self.bias -= self.lr * db


Вывод метрик регрессии

In [95]:
print("Sklearn:", evaluate_regression(y_test_reg, y_pred_sk_lin))
print("Custom :", evaluate_regression(y_test_reg, y_pred_my_lin))

Sklearn: {'MAE': 2.1820470221279193, 'RMSE': np.float64(3.0254235827736173), 'R2': 0.912184095154691}


ValueError: Input contains NaN.

Кастомная реализация использует градиентный спуск, из-за особенностей моего датасета, где признаки имеют разные масштабы (например, Surface_Area — 515-809, а Overall_Height — 3,5–7), он не сходится, значения весов уходят в бесконечность, поэтому появляется NaN. На этапе улучшения мы масштабируем признаки и этой проблемы не будет!

Применим улучшения

Скейлинг

In [96]:
scaler_cls = StandardScaler()
X_train_cls_scaled = scaler_cls.fit_transform(X_train_cls)
X_test_cls_scaled = scaler_cls.transform(X_test_cls)

Запуск улучшенной модели логической регрессии sklearn (классификация)

In [97]:
sk_log_opt = LogisticRegression(max_iter=1000, random_state=42)
sk_log_opt.fit(X_train_cls_scaled, y_train_cls)
y_pred_sk_opt = sk_log_opt.predict(X_test_cls_scaled)

Запуск кастомной улучшенной модели логической регрессии (классификация)

In [98]:
my_log_opt = MyLogisticClassifier(learning_rate=0.1, n_iterations=1000)
my_log_opt.fit(X_train_cls_scaled, y_train_cls)
y_pred_my_opt = my_log_opt.predict(X_test_cls_scaled)

Вывод метрик классификации

In [99]:
print("Sklearn:", evaluate_classification(y_test_cls, y_pred_sk_opt))
print("Custom :", evaluate_classification(y_test_cls, y_pred_my_opt))

Sklearn: {'Accuracy': 0.8097560975609757, 'F1': 0.807243989148881, 'ROC-AUC': 0.807142857142857}
Custom : {'Accuracy': 0.8097560975609757, 'F1': 0.807243989148881, 'ROC-AUC': 0.807142857142857}


Скейлинг

In [100]:
scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

Генерация полиномиальных признаков

In [101]:
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_train_reg_poly = poly.fit_transform(X_train_reg_scaled)
X_test_reg_poly = poly.transform(X_test_reg_scaled)

Скейлинг полиномиальных признаков

In [102]:
scaler_poly = StandardScaler()
X_train_reg_poly_scaled = scaler_poly.fit_transform(X_train_reg_poly)
X_test_reg_poly_scaled = scaler_poly.transform(X_test_reg_poly)

Запуск улучшенной модели линейной регрессии sklearn (регрессия)

In [103]:
sk_lin_poly = LinearRegression()
sk_lin_poly.fit(X_train_reg_poly, y_train_reg)
y_pred_sk_poly = sk_lin_poly.predict(X_test_reg_poly)

Запуск кастомной улучшенной модели линейной регрессии (регрессия)

In [104]:
my_lin_poly = MyLinearRegressor(learning_rate=0.1, n_iterations=3000)
my_lin_poly.fit(X_train_reg_poly_scaled, y_train_reg)
y_pred_my_poly = my_lin_poly.predict(X_test_reg_poly_scaled)

Вывод метрик регрессии

In [105]:
print("Sklearn:", evaluate_regression(y_test_reg, y_pred_sk_poly))
print("Custom :", evaluate_regression(y_test_reg, y_pred_my_poly))

Sklearn: {'MAE': 0.60420014656585, 'RMSE': np.float64(0.8029563871333705), 'R2': 0.9938143588850876}
Custom : {'MAE': 1.6080726898414572, 'RMSE': np.float64(2.2043898036311487), 'R2': 0.9533794286110633}
